In [ ]:
#!pip install torch
#!pip install tqdm
#!pip install torchmetrics

In [ ]:
import numpy as np
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [ ]:
from torch_tools import Trainer

# Classification

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        batch_size, *_ = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        return x

## Data provided as dataloader

In [ ]:
train_set = datasets.MNIST("data/", train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

test_set = datasets.MNIST("data/", train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

In [ ]:
model = MyModel()
trainer = Trainer(model, loss="crossentropy", optimizer="adam", metric="accuracy", device='cuda')

In [ ]:
history = trainer.fit(train_loader, epochs=2, validation_data=test_loader)

In [ ]:
history

In [ ]:
trainer.evaluate(test_loader)

# Regression

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(100, 50)
        self.layer_2 = nn.Linear(50, 10)
        self.layer_3 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        return x.view(len(x),)

## Data provided as numpy

In [ ]:
X, y = make_regression(n_samples=20_000, n_features=100, n_informative=100, n_targets=1, noise=0.2, random_state=101)
X, y = X.astype(np.float32), y.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model = MyModel()
trainer = Trainer(model, loss="mse", optimizer="adam", metric="mae")

In [ ]:
history = trainer.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2)

In [ ]:
history

In [ ]:
trainer.evaluate(X_test, y_test)